In [1]:
from tscode.utils import graphize, read_xyz, write_xyz
from tscode.torsion_module import _get_hydrogen_bonds
from tscode.pt import pt

import py3Dmol



def mol_3D_view(filename, extra_hb_ids, ts_ids, size=(800, 500)):
    """Draw molecule in 3D
    Return:
    ----
        viewer: py3Dmol.view, a class for constructing embedded 3Dmol.js views in ipython notebooks.
    """

    viewer = py3Dmol.view(width=size[0], height=size[1])

    sphere_scale = 0.18
    stick_radius = 0.15

    mol = read_xyz(filename)
    coords = mol.atomcoords[0]
    atomnos = mol.atomnos

    with open(filename, 'r') as f:
        string = f.read()

    viewer.addModel(string, 'xyz')
    viewer.setStyle({"stick":{"radius":stick_radius}, "sphere":{"scale":sphere_scale}})
    viewer.setStyle({"elem":"F"}, {"stick":{"radius":stick_radius, "color":"aquamarine"}, "sphere":{"scale":sphere_scale, "color":"aquamarine"}})
    viewer.setStyle({"elem":"S"}, {"stick":{"radius":stick_radius, "color":"yellow"}, "sphere":{"scale":sphere_scale, "color":"yellow"}})

    graph = graphize(coords, atomnos)
    hydrogen_bonds = _get_hydrogen_bonds(coords, atomnos, graph, max_angle=30)

    if extra_hb_ids is not None:
        hydrogen_bonds += extra_hb_ids

    for i1, i2 in hydrogen_bonds:
        c1 = coords[i1]
        c2 = coords[i2]
        viewer.addCylinder({
            "gapLength" : 0.5,
            "alpha" : 0.85,
            "fromCap" : "round",
            "toCap" : "round",
            "dashed" : True,
            "color" : "#ffc387",
            "start" : {"x":c1[0],"y":c1[1],"z":c1[2]},
            "end":{"x":c2[0],"y":c2[1],"z":c2[2]},
            })
        
    if ts_ids is not None:
        for i1, i2 in ts_ids:
            c1 = coords[i1]
            c2 = coords[i2]
            viewer.addCylinder({
                "alpha" : 0.5,
                "radius":stick_radius,
                "color" : "#ffc387",
                "start" : {"x":c1[0],"y":c1[1],"z":c1[2]},
                "end":{"x":c2[0],"y":c2[1],"z":c2[2]},
                })
            
    viewer.setProjection("orthographic")
    viewer.enableFog(True)
    viewer.zoomTo()
    viewer.zoom(1.5)

    return viewer

# mol = read_xyz('mol1.xyz')


In [2]:
# from ipywidgets import interact, fixed, IntSlider

xyz_traj = read_xyz('mol2.xyz')

In [ ]:
def get_xyz_string(coords, atomnos):
    string = str(len(coords))
    string += f'\ntemp\n'
    for i, atom in enumerate(coords):
        string += '%s     % .6f % .6f % .6f\n' % (pt[atomnos[i]].symbol, atom[0], atom[1], atom[2])
    return string

def movie_3D_view(filename, extra_hb_ids=None, ts_ids=None, size=(800, 500)):
    """Draw molecule in 3D
    Return:
    ----
        viewer: py3Dmol.view, a class for constructing embedded 3Dmol.js views in ipython notebooks.
    """

    viewer = py3Dmol.view(width=size[0], height=size[1])

    mol = read_xyz(filename)
    atomnos = mol.atomnos
    graph = graphize(mol.atomcoords[0], atomnos)

    with open(filename, "r") as f:
        string = f.read()

    viewer.addModelsAsFrames(string, 'xyz')

    for c, coords in enumerate(mol.atomcoords):

        coords = mol.atomcoords[c]


        hydrogen_bonds = _get_hydrogen_bonds(coords, atomnos, graph, max_angle=30)

        if extra_hb_ids is not None:
            hydrogen_bonds += extra_hb_ids

        for i1, i2 in hydrogen_bonds:
            c1 = coords[i1]
            c2 = coords[i2]
            viewer.addCylinder({
                "frame" : c,
                "gapLength" : 0.5,
                "alpha" : 0.85,
                "fromCap" : "round",
                "toCap" : "round",
                "dashed" : True,
                "color" : "#ffc387",
                "start" : {"x":c1[0],"y":c1[1],"z":c1[2]},
                "end":{"x":c2[0],"y":c2[1],"z":c2[2]},
                })
        
        if ts_ids is not None:
            for i1, i2 in ts_ids:
                c1 = coords[i1]
                c2 = coords[i2]
                viewer.addCylinder({
                    "frame" : c,
                    "alpha" : 0.5,
                    "radius":stick_radius,
                    "color" : "#ffc387",
                    "start" : {"x":c1[0],"y":c1[1],"z":c1[2]},
                    "end":{"x":c2[0],"y":c2[1],"z":c2[2]},
                    })

    sphere_scale = 0.18
    stick_radius = 0.15
                
    viewer.setStyle({"stick":{"radius":stick_radius}, "sphere":{"scale":sphere_scale}})
    viewer.setStyle({"elem":"F"}, {"stick":{"radius":stick_radius, "color":"aquamarine"}, "sphere":{"scale":sphere_scale, "color":"aquamarine"}})
    viewer.setStyle({"elem":"S"}, {"stick":{"radius":stick_radius, "color":"yellow"}, "sphere":{"scale":sphere_scale, "color":"yellow"}})
            
    viewer.setProjection("orthographic")
    viewer.enableFog(True)
    viewer.zoomTo()
    viewer.zoom(1.5)
    viewer.animate({"loop":"backAndForth", "interval":100}) # or "backAndForth", "reps":1, 'step': 1

    return viewer

v = movie_3D_view("folding_anim2.xyz", size=(650, 400))
v.rotate(60, "x")
v.show()

In [ ]:
## MAKE A DROPDOWN VIEWER
from ipywidgets import interact, Dropdown
from ipywidgets.embed import embed_minimal_html
import numpy as np

dropdown = {
    "TS1 Sa anti" : ("TS1_Sa_anti_folded_ether_TS.xyz", [(9, 137), (95, 107)], [(6, 14)]),
    "TS1 Ra syn" : ("TS1_Ra_syn_hairpin_flipped_TS.xyz", None, [(6, 14)]),
    "TS2 Sa anti" : ("TS2_Sa_anti_folded_flip_TS.xyz", [(7, 133)], [(6, 7)]),
    "TS2 Ra syn" : ("TS2_Ra_syn_hairpin_TS.xyz", None, [(6, 7)]),
}

def structures_dropdown(name):
    filename, extra_hb_ids, ts_ids = dropdown[name]
    mol = read_xyz(filename)
    viewer = mol_3D_view(mol.atomcoords[0], mol.atomnos, extra_hb_ids=extra_hb_ids, ts_ids=ts_ids)
    viewer.zoom(1.8)

    # label_center = np.mean(mol.atomcoords[0][np.array(ts_ids[0])], axis=0)
    # print(label_center)
    # viewer.addLabel(name, {"position":{"x":label_center[0],"y":label_center[1],"z":label_center[2]}})
    return viewer.show()

interact(structures_dropdown, name=Dropdown(options=dropdown.keys(),
                                            value="TS1 Sa anti",
                                            description="Structure: "))


In [12]:
def mol_3D_view_grid(mols, size=(800, 500)):
    """Draw molecule in 3D
    Return:
    ----
        viewer: py3Dmol.view, a class for constructing embedded 3Dmol.js views in ipython notebooks.
    """

    assert len(mols.keys()) == 4

    viewer = py3Dmol.view(width=size[0], height=size[1], viewergrid=(2,2), linked=False)

    sphere_scale = 0.18
    stick_radius = 0.15

    for name, window in zip(mols, ((0,0), (0,1), (1,0), (1,1))):

        filename, extra_hb_ids, ts_ids = mols[name]
        mol = read_xyz(filename)
        coords = mol.atomcoords[0]
        atomnos = mol.atomnos

        viewer.addModel(get_xyz_string(coords, atomnos), 'xyz', viewer=window)
        viewer.setStyle({"stick":{"radius":stick_radius}, "sphere":{"scale":sphere_scale}})
        viewer.setStyle({"elem":"F"}, {"stick":{"radius":stick_radius, "color":"aquamarine"}, "sphere":{"scale":sphere_scale, "color":"aquamarine"}})
        viewer.setStyle({"elem":"S"}, {"stick":{"radius":stick_radius, "color":"yellow"}, "sphere":{"scale":sphere_scale, "color":"yellow"}})

        graph = graphize(coords, atomnos)
        hydrogen_bonds = _get_hydrogen_bonds(coords, atomnos, graph, max_angle=30)

        if extra_hb_ids is not None:
            hydrogen_bonds += extra_hb_ids

        for i1, i2 in hydrogen_bonds:
            c1 = coords[i1]
            c2 = coords[i2]
            viewer.addCylinder({
                "gapLength" : 0.5,
                "alpha" : 0.85,
                "fromCap" : "round",
                "toCap" : "round",
                "dashed" : True,
                "color" : "#ffc387",
                "start" : {"x":c1[0],"y":c1[1],"z":c1[2]},
                "end":{"x":c2[0],"y":c2[1],"z":c2[2]},
                }, viewer=window)
            
        if ts_ids is not None:
            for i1, i2 in ts_ids:
                c1 = coords[i1]
                c2 = coords[i2]
                viewer.addCylinder({
                    "alpha" : 0.5,
                    "radius":stick_radius,
                    "color" : "#ffc387",
                    "start" : {"x":c1[0],"y":c1[1],"z":c1[2]},
                    "end":{"x":c2[0],"y":c2[1],"z":c2[2]},
                    }, viewer=window)
            
    viewer.setProjection("orthographic")
    viewer.enableFog(True)
    viewer.zoomTo()
    viewer.zoom(1.5)

    return viewer

mols = {
    "TS1 Sa anti" : ("TS1_Sa_anti_folded_ether_TS.xyz", [(9, 137), (95, 107)], [(6, 14)]),
    "TS1 Ra syn" : ("TS1_Ra_syn_hairpin_flipped_TS.xyz", None, [(6, 14)]),
    "TS2 Sa anti" : ("TS2_Sa_anti_folded_flip_TS.xyz", [(7, 133)], [(6, 7)]),
    "TS2 Ra syn" : ("TS2_Ra_syn_hairpin_TS.xyz", None, [(6, 7)]),
}

mol_3D_view_grid(mols).show()


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [33]:
a, b, c = mols["TS1 Sa anti"]
v = mol_3D_view(
            a,
            extra_hb_ids=b, 
            ts_ids=c,
            size=(650,400)
        )
v.rotate(-20, "x")
v.rotate(20, "y")
v.rotate(180, "z")


with open("TS1_Sa_sub.xyz", 'r') as f:
    substrate = f.read()

v.addModel(substrate, "xyz")
v.addStyle({"model":1}, {"stick":{"radius":0.3, "opacity":0.5, "color":"#ffc387"}, "sphere":{"scale":0.3, "opacity":0.5, "color":"#ffc387"}})


v.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [34]:
a, b, c = mols["TS1 Ra syn"]
v = mol_3D_view(
            a,
            extra_hb_ids=b, 
            ts_ids=c,
            size=(650,400)
        )
v.rotate(120, "x")
v.rotate(-30, "y")
# v.rotate(90, "z")
# v.rotate(-90, "y")


with open("TS1_Ra_sub.xyz", 'r') as f:
    substrate = f.read()

v.addModel(substrate, "xyz")
v.addStyle({"model":1}, {"stick":{"radius":0.3, "opacity":0.5, "color":"#ffc387"}, "sphere":{"scale":0.3, "opacity":0.5, "color":"#ffc387"}})


v.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [35]:
a, b, c = mols["TS2 Sa anti"]
v = mol_3D_view(
            a,
            extra_hb_ids=b, 
            ts_ids=c,
            size=(650,400)
        )
v.rotate(180, "x")
# v.rotate(-30, "y")
# v.rotate(90, "z")
# v.rotate(-90, "y")

with open("TS2_Sa_sub.xyz", 'r') as f:
    substrate = f.read()

v.addModel(substrate, "xyz")
v.addStyle({"model":1}, {"stick":{"radius":0.3, "opacity":0.5, "color":"#ffc387"}, "sphere":{"scale":0.3, "opacity":0.5, "color":"#ffc387"}})


v.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [36]:
a, b, c = mols["TS2 Ra syn"]
v = mol_3D_view(
            a,
            extra_hb_ids=b, 
            ts_ids=c,
            size=(650,400)
        )
v.rotate(120, "x")
v.rotate(-30, "y")
v.rotate(30, "z")
# v.rotate(-90, "y")

with open("TS2_Ra_sub.xyz", 'r') as f:
    substrate = f.read()

v.addModel(substrate, "xyz")
v.addStyle({"model":1}, {"stick":{"radius":0.3, "opacity":0.5, "color":"#ffc387"}, "sphere":{"scale":0.3, "opacity":0.5, "color":"#ffc387"}})


v.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol